## Imports and Settings

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import squareform, cdist
import time

In [2]:
path_to_files = "./../Dataset/dsC7O2H10nsd.xyz/"
filenames = os.listdir(path_to_files)
print('number of files: {}'.format(len(filenames)))

number of files: 6095


## Create a Dataframe from Input files

In [3]:
list_ = []
for file in filenames:
    filepath = os.path.join(path_to_files, file)
    try:
        df_single = pd.read_csv(filepath, skiprows=2,
                               skipfooter=3, delimiter='\t',
                               names=['atomtype', 'x', 'y', 'z', 'charge'], 
                               dtype=dict(atomtype=str, x=float, y=float, z=float, charge=float))
    except:
        print(file)
    df_single['file'] = file
    list_.append(df_single)
df_all = pd.concat(list_)
df_all.head(5)

/Users/stfn/anaconda3/envs/fproj/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


dsC7O2H10nsd_0249.xyz
dsC7O2H10nsd_2258.xyz
dsC7O2H10nsd_2556.xyz


,atomtype,x,y,z,charge,file
0,C,-1.839613,0.529296,3.182068,-0.414977,dsC7O2H10nsd_0001.xyz
1,C,-2.146536,-0.164467,1.842511,0.232941,dsC7O2H10nsd_0001.xyz
2,C,-3.586600,0.132659,1.386682,-0.415379,dsC7O2H10nsd_0001.xyz
3,O,-1.945267,-1.573642,1.958069,-0.461014,dsC7O2H10nsd_0001.xyz
4,C,-1.164462,0.268959,0.791005,0.388553,dsC7O2H10nsd_0001.xyz


## Prepare raw Data for Transformation

In [4]:
n_atoms = 19
# sort by file and atomtype
df_all = df_all.sort_values(['file', 'atomtype']).reset_index(drop=True)
# create file id column
df_all['file_id'] = (df_all.index) // n_atoms + 1

In [5]:
df_all.head(5)

,atomtype,x,y,z,charge,file,file_id
0,C,-1.839613,0.529296,3.182068,-0.414977,dsC7O2H10nsd_0001.xyz,1
1,C,-2.146536,-0.164467,1.842511,0.232941,dsC7O2H10nsd_0001.xyz,1
2,C,-3.586600,0.132659,1.386682,-0.415379,dsC7O2H10nsd_0001.xyz,1
3,C,-1.164462,0.268959,0.791005,0.388553,dsC7O2H10nsd_0001.xyz,1
4,C,-0.247116,-0.397562,0.042898,-0.278918,dsC7O2H10nsd_0001.xyz,1


## Transform Dataframe to Numpy Array for faster Calculations

In [6]:
raw_matrix = df_all[['file_id', 'atomtype', 'x', 'y', 'z', 'charge']].values

## Transformation Functions

In [14]:
def get_spherical(positions):
    """
    Transform 3D cartesian coordinates to spherical coordinates that can
    be used for the nural network input.
    
    Parameters
    ----------
    positions : ndarray
        Array of shape (N, 3) where N is the number of coordinates that
        needs to be transformed.
    
    Returns
    -------
    ndarray
        Array of shape (N, 4) where N is the number of transformed coordinates.
        Transformed coordinates for one position: (1/r, cos(theta), cos(phi), sin(phi)).
        
    """            
    positions = positions.astype(float)
    r = np.linalg.norm(positions, axis=1)
    theta = np.arcsin(positions[:, 2]/r)
    phi = np.arctan2(positions[:, 0], positions[:, 1])
    return np.append(1/r, np.cos(theta), np.cos(phi), np.sin(phi), axis=0)

In [15]:
def change_base(positions, x, y, z, o):
    """
    Calculate the base transformation from the standard basis to the new axes x, y, z.
    
    Parameters
    ----------
    positions : np.array
        3D atom position in the standard basis
    x : np.array
        new x-axis
    y : np.array
        new y-axis
    z : np.array
        new z-axis    
    o : np.array
        new origin
    
    Returns
    -------
    new_positions : np.array
        3D atom position in the new basis.
        Same shape as positions.

    """
    positions -= o
    basis = np.vstack((x, y, z))
    basis_inv = np.linalg.inv(basis)
    new_positions = basis_inv.dot(positions.T).T
    return new_positions


In [16]:
def get_input_data(raw_matrix):
    """
    Calculate the training input for the sub-networks from a given molecular configuration.

    Parameters
    ----------
    raw_matrix : np.array
        Matrix of the raw input data for all files and all atoms
    raw_matrix_cols : list
        Column names for the raw_matrix
    
    Returns
    -------
    X : np.array
        Training data with 'atomtype' and 'relative position-vector' for all other atoms.
    Y : np.array
        Training labels (Mullikan Charge)

    """
    file_id = 0
    atomtype = 1
    x = 2
    y = 3
    z = 4
    charge = 5
    # make a copy
    network_inputs = []
    # create a column for the pos vector
    start = 0
    for i in np.unique(raw_matrix[:, 0]):
        stop = i*19
        molecule = raw_matrix[start:stop]
        start = stop
        dist_matrix = cdist(molecule[:, 2:5], molecule[:, 2:5])
        for atom in range(len(molecule)):
            zero = molecule[atom, 2:5].astype(float)
            one_id, two_id = dist_matrix[atom].argsort()[1:3]
            one = molecule[one_id, 2:5].astype(float)
            two = molecule[two_id, 2:5].astype(float)
            new_x = one - zero
            new_z = np.cross(new_x, two-zero)
            new_y = np.cross(new_x, new_z)
            new_x /= np.linalg.norm(new_x)
            new_y /= np.linalg.norm(new_y)
            new_z /= np.linalg.norm(new_z)
            transformed_positions = change_base(molecule[:, 2:5],
                                        new_x, new_y, new_z,
                                        zero)
            transformed_positions = np.delete(transformed_positions,
                                              (atom), axis=0)
            transformed_positions_sph = get_spherical(transformed_positions)
            network_inputs.append((molecule[atom, 1], transformed_positions_sph, molecule[atom, 5]))
            
    return network_inputs

## Run Calculations

In [17]:
start = time.time()
network_in = get_input_data(raw_matrix)
print('time: {}'.format(time.time()-start))

TypeError: append() got multiple values for argument 'axis'

In [13]:
network_in

[('C', [array([ 0.64958067,  0.39427886,  0.40029509,  0.27472635,  0.21970049,
           0.23799322,  0.91206262,  0.91507955,  0.91470701,  0.36290334,
           0.35473702,  0.28712603,  0.38557035,  0.24442908,  0.17929915,
           0.20284602,  0.41059255,  0.33504372]),
   array([ 0.4646224 ,  0.4644106 ,  0.56519476,  0.73153564,  0.73566731,
           0.70493829,  0.88929119,  0.81311576,  0.99390287,  0.77833562,
           0.47668396,  0.40629406,  0.9586766 ,  0.81442753,  0.7743416 ,
           0.74688428,  0.87481235,  0.61282585]),
   array([ 0.90501698,  0.58402366, -0.06744293,  0.09862222, -0.27113545,
          -0.63578808,  0.5480902 ,  0.04323418, -0.96089209,  0.56781328,
          -0.23964095,  0.85337956,  0.99838721,  0.37155976, -0.2320506 ,
          -0.79241739,  0.92253136, -0.76821426]),
   array([ 0.42537544, -0.81173664,  0.99772313,  0.99512495,  0.9625412 ,
           0.77186366, -0.83641923,  0.99906497, -0.27692306, -0.82315738,
          -0.9708

## Use Pandas to Save the network inputs to hdf5

In [11]:
df_input = pd.DataFrame(network_in, columns=['atomtype', 'coordinates', 'charge'])
df_c = df_input.loc[df_input.atomtype=='C']
df_o = df_input.loc[df_input.atomtype=='O']
df_h = df_input.loc[df_input.atomtype=='H']

In [12]:
df_c.to_hdf('../Dataset/network_inputs.h5', 'ctype')
df_o.to_hdf('../Dataset/network_inputs.h5', 'otype')
df_h.to_hdf('../Dataset/network_inputs.h5', 'htype')

/Users/stfn/anaconda3/envs/fproj/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['atomtype', 'coordinates']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
